In [32]:
import bittensor
from tqdm import tqdm
import numpy as np

sub = bittensor.subtensor(network="wss://lite.chain.opentensor.ai:443")

In [8]:
arb_blocks_map = sub.query_map_subtensor(
    name="ColdkeyArbitrationBlock",
)

In [9]:
import requests
import json
import re

pat = re.compile(r'class="break-words text-lg line-clamp-1">(5\w{47})')
pat_block = re.compile(r"(33\d{5})")
url = "https://keycheck.taostats.io/"
def get_destination_and_block(old_coldkey):
    url_ = url + old_coldkey
    response = requests.get(url_)
    response_text = response.text # html
    addy = pat.findall(response_text)[0]

    block = pat_block.findall(response_text)[0]
    return addy, block

In [ ]:
curr_block = sub.get_current_block()
moved_already = {}
for old_coldkey, arb_block in arb_blocks_map:
    arb_block = arb_block.value
    old_coldkey = old_coldkey.value

    dest, block = get_destination_and_block(old_coldkey)
    if arb_block <= curr_block + 250 and arb_block >= 3_372_000:
        print(f"Old coldkey: {old_coldkey}, new coldkey: {dest}, block: {arb_block}")
        moved_already[old_coldkey] = dest, block, arb_block

In [25]:
with open("accts.json", "r") as f:
    jake_accts = json.load(f)

In [36]:
import substrateinterface
substrate: substrateinterface.SubstrateInterface

zero_weight = {
    "ref_time": 0,
    "proof_size": 0,
}
with sub.substrate as substrate:
    
    calls = []
    for acct in jake_accts:
        src = acct["source"]
        dst = acct["dest"][0]
        
        swap_call = substrate.compose_call(
            call_module='SubtensorModule',
            call_function='sudo_hotfix_swap_coldkey_delegates',
            call_params={
                "old_coldkey": src,
                "new_coldkey": dst,
            }
        )
        calls.append(swap_call)

    batch_call = substrate.compose_call(
        call_module='Utility',
        call_function='batch',
        call_params={
            "calls": calls,
        }
    )

    sudo_call = substrate.compose_call(
        call_module='Sudo',
        call_function='sudo_unchecked_weight',
        call_params={
            "call": batch_call,
            "weight": zero_weight
        }
    )


    with open("sudo_hash.txt", "w") as f:
        f.write(str(sudo_call.call_hash.hex()))

    print(sudo_call.call_hash.hex())


6b9f3c74794d15e790d48892df7faee318ef1abce35b44ec8c0ca5cc3bb31263


In [38]:
with sub.substrate as substrate:
    multi_sig_call = substrate.compose_call(
        call_module='Multisig',
        call_function='as_multi',
        call_params={
            "call": sudo_call,
            "threshold": 2,
            "other_signatories": [
                "5FPJHiZnj5wk2hGxer7xwBJi91WNwsssYeVspq7Sd9mhwyxt",
                "5GmX9ZESxY6b8N2LxWKnaYH3VmbmPRjM4iu5f8oRs78R2Hzi"
            ],
            "max_weight": zero_weight,
            "maybe_timepoint": None,
        }
    )

    w = bittensor.wallet(name="trium_sig", hotkey="h0")
    signed = substrate.create_signed_extrinsic(
        multi_sig_call,
        w.coldkey
    )

Output()

In [39]:
signed

<GenericExtrinsic(value={'account_id': '0xacc85abe267657a12aa2795fffb54618b92e83fb64f5930ae3a4c7a17513215b', 'signature': {'Sr25519': '0x22404fc136f84792edcf5147b2e8e56275705706233c8b1c8a7f33dcf2906e442d3aa0f36448fde7432e1fa407b17a8bba2b2500f06187252e7d17aae272128f'}, 'call_function': 'as_multi', 'call_module': 'Multisig', 'call_args': {'call': <GenericCall(value={'call_module': 'Sudo', 'call_function': 'sudo_unchecked_weight', 'call_args': {'call': <GenericCall(value={'call_module': 'Utility', 'call_function': 'batch', 'call_args': {'calls': [<GenericCall(value={'call_module': 'SubtensorModule', 'call_function': 'sudo_hotfix_swap_coldkey_delegates', 'call_args': {'old_coldkey': '5CQ2X83y5HijGX5h2HaA3ecYZo4nRXxS2XgN8TgLwkEUkXvW', 'new_coldkey': '5GP7fVa4tyiXkBpoxquJ2Hua19nvcqTE7FpyNNdpgBp741qk'}})>, <GenericCall(value={'call_module': 'SubtensorModule', 'call_function': 'sudo_hotfix_swap_coldkey_delegates', 'call_args': {'old_coldkey': '5EA1W6o97C6G8uhawC5MxHrYtiE7bTSsrA9Rg2uRbb4eWUF8',

In [19]:
with open("moved_already.json", "r") as f:
    moved_already = json.load(f)

In [20]:
with open("accts.json", "r") as f:
    jake_accts = json.load(f)

for acct in jake_accts:
    src = acct["source"]
    dst = acct["dest"][0]
    
    if src in moved_already:
        if dst != moved_already[src][0]:
            print("Mismatch", src, dst, moved_already[src][0])
    else:
        print("Not in moved already", src, dst)


Not in moved already 5Ctwns1p4UNCvESjVkqcvn8HAFEgXADPigLnN5tv1pQeVGH5 5E812kkV5ZWavVi9N5tL7mwQrtfBCcj7XXktNQrB5bgBu6Aq
Not in moved already 5GqMtSsjMKsrcMnhQTJi4yyTS6XDS47PLy6vYioNWTdgpUWJ 5Fc4JtKKCQ6iCmBAEdxbnJ3Liu81qCgt7bxCR6sJq9JQKeZH
Not in moved already 5G9Qm8fWx5MFQ6GrXyqXbiZr9PaJouYJNYzbhJsFscXEV8Rg 5Ec85FDXq2bfWvfjwdjifMycXCcJe7rJfm9WG4YkWTbk9w1Q
Not in moved already 5CaW1fvjxjbgevmWcXggZUfocxbKPF8MqFsVFmYgcQUJ9y33 5FCKdrQhS9jikJW1KkBNBrEwjq733VGbEFL8K9xJQ3p3jrY8
Not in moved already 5Gsm3etzwwJDSvv1F1Hq29uj5KQWpFfd7HBAvUAq7LGjkCtX 5CJ4fNN3vPHPKU7GeDdUVHqrdfpYcakA1eH62BqehavSuPw9
Not in moved already 5FCwQ83R6MpZFHntgLmPrDKb3juCdgrAj7iWeFwdv5naKiat 5HDijFsM1oSXp6uewWNLr9XXdyr6TDx7k3V8XfVRUpUaAhsA
Not in moved already 5EspT5sJwLe7yTAvMfWgRxBChuCfBygV2stebnCu3HAvEg9T 5DnSqpyLaGJ35cbrEXfaDKGWdihRFh9439Q6pKf7yswhxVzQ
Not in moved already 5EJRDNGGnSf3YfpUz6xrM3gkvLhNPDJ7QcK2mK48gZUEGH3r 5HjCynhYoYhUzC7RHkKrdq9VfykXCkZPQwZhfeeK3CF5M9p5
Not in moved already 5EAZ5d6Mi9KwngzSsDLTd9xGktn

In [21]:
len(moved_already), len(jake_accts)

(122, 131)

In [104]:
len(moved_already)

122

In [111]:
for old_coldkey, (dest, block, arb_block) in moved_already.items():
    result = sub.query_subtensor(
        name="ColdkeySwapDestinations",
        params=[old_coldkey],
        block=arb_block,
    )
    if result is not None:
        print(result[0], dest, result[0] == dest) 

5Gn9nv7Le5XYitfP86bdDuc92fUtcPGQtDM6vCcZur4fwUpy 5Gn9nv7Le5XYitfP86bdDuc92fUtcPGQtDM6vCcZur4fwUpy True
5DXCTBN6dqt1cSxDFWxtkHWnJcL5VdKgDVPzBygihCqq3HGD 5DXCTBN6dqt1cSxDFWxtkHWnJcL5VdKgDVPzBygihCqq3HGD True
5HTRDSNRbx3cLqUh6maywq7p2Rvx71gX5RKFPmL53tsRvDFV 5HTRDSNRbx3cLqUh6maywq7p2Rvx71gX5RKFPmL53tsRvDFV True
5FFXXq9KN2BYZyydt15D2HoY5UhEHCnVFVsDstV8J6hTnYRM 5FFXXq9KN2BYZyydt15D2HoY5UhEHCnVFVsDstV8J6hTnYRM True
5CoMgHTkpS24aV6J8BMxVaLnbm88FYkhYHVGMNhcY28PUoUK 5CoMgHTkpS24aV6J8BMxVaLnbm88FYkhYHVGMNhcY28PUoUK True
5Dz2dxSZD1e4qvsUyjE5njDp4D5SiMHj3giQKWzALkfzo8WV 5Dz2dxSZD1e4qvsUyjE5njDp4D5SiMHj3giQKWzALkfzo8WV True
5EFJDS6AwWUW2grA1Jkvm6HTWHVg5L8Y9V6Tnn12n84E3kPH 5EFJDS6AwWUW2grA1Jkvm6HTWHVg5L8Y9V6Tnn12n84E3kPH True
5EcZsewZSTxUaX8gwyHzkKsqT3NwLP1n2faZPyjttCeaPdYe 5EcZsewZSTxUaX8gwyHzkKsqT3NwLP1n2faZPyjttCeaPdYe True
5CqichJPQkVAigfEohnvVDEUUWZjsevBjmd2RMeLdUyoT5nK 5CqichJPQkVAigfEohnvVDEUUWZjsevBjmd2RMeLdUyoT5nK True
5HpHbavJK2CJNPXaHXGcGfzJ9eMpDDbAdw43YBscNVDLNn3i 5HpHbavJK2CJNPXaHXGcGfzJ

In [103]:
with open("moved_already.json", "w") as f:
    json.dump(moved_already, f)